In [1]:
import sys, json, re, collections
import pandas as pd
from pathlib import Path
from decouple import config
sys.path.append("../src/")
from llm_helpers import openai_client

pd.set_option('display.max_columns', 1000, 'display.width', 1000, 'display.max_rows',1000)

data_dir = Path(".").absolute().parent/"data"
ls = lambda p:print("\n".join(map(str,p.iterdir())))

ls(data_dir)

/Users/dimid/llm_workshop/data/sample_apps.parquet


In [2]:
df = pd.read_parquet(data_dir / "sample_apps.parquet").sample(9)
df

,bundle_id,title,description,store_url,category_names,ios
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False


In [3]:
categories = df["category_names"].str.lower().str.split(',').explode().value_counts()
categories

category_names
application           3
games                 3
game                  3
entertainment         2
game_puzzle           2
shopping              1
lifestyle             1
strategy              1
action                1
news_and_magazines    1
adventure             1
casual                1
puzzle                1
simulation            1
game_word             1
Name: count, dtype: int64

## Naive approach, just Ask nicely.

In [4]:
def openai_ask(prompts):
    response = openai_client.completions.create(
        model="text-davinci-003",
        prompt=prompts,
    )
    ret = [choice.text.strip().lower() for choice in response.choices]
    return ret


In [5]:
prompt = "please choose the most likely category that apply to 'Crossword Jam' from the following list:\n"
prompt+="\n".join(categories.index)
openai_ask(prompt)

['game_puzzle']

In [6]:
openai_ask(prompt)

['game_puzzle']

In [7]:
openai_ask(prompt)

['game_puzzle']

Seem to work pretty, well - let's try multi label?

In [8]:
prompt = "please choose all the categories that apply to 'Crossword Jam' from the following list:\n"
prompt+="\n".join(categories.index)
[sorted(map(str.strip,l.strip().replace(",", "\n").split("\n"))) for l in openai_ask([prompt]*10)]

[['application', 'entertainment', 'game', 'game_', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games', 'l'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games'],
 ['application', 'entertainment', 'game', 'game_puzzle', 'games', 'l']]

In [9]:
"game casual" in categories

False

## Using function calls
### Choose most likely class

In [10]:
def classify_most_likely(prompt):
    messages = [{"role": "user", "content": prompt}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "classify_app",
                "description": "Classify to an enum type",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "class": {"type": "string", "enum": list(categories.index)},
                    },
                    "required": ["class"],
                },
            },
        }
    ]
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "classify_app"}}
    )
    ret =  response.choices[0].message.tool_calls[0].function.arguments
    return json.loads(ret)["class"]

classify_most_likely("please choose the most likely category that apply to 'Crossword Jam'")

'game_word'

In [11]:
def classify_multiclass(prompt):
    messages = [{"role": "user", "content": prompt}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "classify_app",
                "description": "Classify to an enum type",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "classes": {"type": "array", "items": {"type": "string", "enum": list(categories.index)}},
                    },
                    "required": ["classes"],
                },
            },
        }
    ]
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "classify_app"}}
    )
    ret =  response.choices[0].message.tool_calls[0].function.arguments
    return json.loads(ret)["classes"]

classify_multiclass("please choose the all the categories that apply to 'Crossword Jam'")

['game', 'puzzle', 'game_word']

In [12]:
classify_multiclass("please choose the all the categories that apply to 'Crossword Jam'")

['application', 'game', 'game_puzzle']

## Question:
Please write ask the model to classify all of the apps using the methods we learnt.

Then create an additional column "jaccard" indicating whether intersection over union of the actual categories and the predicted ones.

Which method was most successful?

In [37]:
def openai_classify_multilabel_naive(desc):
    prompt = f"""please choose all the categories that apply to an app with the following description: '{desc}'. 

    Choose only categories from the following list:\n"""
    prompt+="\n".join(categories.index)

    choice = openai_ask(prompts=prompt)[0] 
    return choice.replace('.', '').replace(",", "\n").split("\n")

df['openai_cats_naive'] = df.description.apply(openai_classify_multilabel_naive)

In [34]:
categories_l = list(categories.index)
categories_l

['application',
 'games',
 'game',
 'entertainment',
 'game_puzzle',
 'shopping',
 'lifestyle',
 'strategy',
 'action',
 'news_and_magazines',
 'adventure',
 'casual',
 'puzzle',
 'simulation',
 'game_word']

In [38]:
df['openai_cats_naive_filtered'] = df.openai_cats_naive.apply(lambda x: [x for x in x if x in categories_l])
df

,bundle_id,title,description,store_url,category_names,ios,openai_cats_naive,openai_cats_naive_filtered
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,"[arcade, role_playing, education, casino, sports]",[]
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False,"[lifestyle, shopping, casual, simulation, p]","[lifestyle, shopping, casual, simulation]"
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,[],[]
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,"[news_and_magazines, lifestyle, strategy, action]","[news_and_magazines, lifestyle, strategy, action]"
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True,"[application, games, entertainment, strateg...",[application]
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,"[games, game, entertainment, game puzzle, ...",[games]
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[game, game_puzzle, action, adventure, casual]","[game, game_puzzle, action, adventure, casual]"
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[application, games, game, entertainment, game...","[application, games, game, entertainment, game..."
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False,"[educational, , application, games, game, ente...","[application, games, game, entertainment]"


In [41]:
df['openai_cats_tool'] = df.description.apply(lambda x: classify_multiclass(f"please choose the all the categories that apply to an app whose description is '{x}'"))

In [42]:
df['openai_cats_tool_filtered'] = df.openai_cats_tool.apply(lambda x: [x for x in x if x in categories_l])

In [43]:
df

,bundle_id,title,description,store_url,category_names,ios,openai_cats_naive,openai_cats_naive_filtered,openai_cats_tool_filtered,openai_cats_tool
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,"[arcade, role_playing, education, casino, sports]",[],"[application, shopping, lifestyle]","[application, shopping, lifestyle]"
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False,"[lifestyle, shopping, casual, simulation, p]","[lifestyle, shopping, casual, simulation]","[application, lifestyle, shopping, adventure, ...","[application, lifestyle, shopping, adventure, ..."
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,[],[],"[application, games, game, strategy, action]","[application, games, game, strategy, action]"
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,"[news_and_magazines, lifestyle, strategy, action]","[news_and_magazines, lifestyle, strategy, action]","[application, news_and_magazines]","[application, news_and_magazines]"
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True,"[application, games, entertainment, strateg...",[application],"[game, strategy, action, adventure, casual, si...","[game, strategy, action, adventure, casual, si..."
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,"[games, game, entertainment, game puzzle, ...",[games],"[application, games, game, entertainment, game...","[application, games, game, entertainment, game..."
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[game, game_puzzle, action, adventure, casual]","[game, game_puzzle, action, adventure, casual]","[application, game, puzzle, adventure, casual,...","[application, game, puzzle, adventure, casual,..."
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, puzzle, casual, strategy]"
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False,"[educational, , application, games, game, ente...","[application, games, game, entertainment]","[application, games, game, entertainment, game...","[application, games, game, entertainment, game..."


In [44]:
df['categories_normalized'] = df.category_names.str.lower().str.split(',')
df

,bundle_id,title,description,store_url,category_names,ios,openai_cats_naive,openai_cats_naive_filtered,openai_cats_tool_filtered,openai_cats_tool,categories_normalized
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,"[arcade, role_playing, education, casino, sports]",[],"[application, shopping, lifestyle]","[application, shopping, lifestyle]","[shopping, application]"
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False,"[lifestyle, shopping, casual, simulation, p]","[lifestyle, shopping, casual, simulation]","[application, lifestyle, shopping, adventure, ...","[application, lifestyle, shopping, adventure, ...","[lifestyle, application]"
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,[],[],"[application, games, game, strategy, action]","[application, games, game, strategy, action]","[games, entertainment, strategy, action]"
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,"[news_and_magazines, lifestyle, strategy, action]","[news_and_magazines, lifestyle, strategy, action]","[application, news_and_magazines]","[application, news_and_magazines]","[news_and_magazines, application]"
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True,"[application, games, entertainment, strateg...",[application],"[game, strategy, action, adventure, casual, si...","[game, strategy, action, adventure, casual, si...","[games, adventure, casual]"
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,"[games, game, entertainment, game puzzle, ...",[games],"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[games, puzzle, entertainment, simulation]"
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[game, game_puzzle, action, adventure, casual]","[game, game_puzzle, action, adventure, casual]","[application, game, puzzle, adventure, casual,...","[application, game, puzzle, adventure, casual,...","[game_puzzle, game]"
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, game]"
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False,"[educational, , application, games, game, ente...","[application, games, game, entertainment]","[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_word, game]"


In [45]:
def jaccard(a, b):
    a_set = set(a)
    b_set = set(b)

    return len(a_set & b_set) / len(a_set | b_set)

df['jaccard_naive'] = df.apply(lambda r: jaccard(r.categories_normalized, r.openai_cats_naive_filtered), axis=1)

In [46]:
df

,bundle_id,title,description,store_url,category_names,ios,openai_cats_naive,openai_cats_naive_filtered,openai_cats_tool_filtered,openai_cats_tool,categories_normalized,jaccard_naive
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,"[arcade, role_playing, education, casino, sports]",[],"[application, shopping, lifestyle]","[application, shopping, lifestyle]","[shopping, application]",0.00
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False,"[lifestyle, shopping, casual, simulation, p]","[lifestyle, shopping, casual, simulation]","[application, lifestyle, shopping, adventure, ...","[application, lifestyle, shopping, adventure, ...","[lifestyle, application]",0.20
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,[],[],"[application, games, game, strategy, action]","[application, games, game, strategy, action]","[games, entertainment, strategy, action]",0.00
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,"[news_and_magazines, lifestyle, strategy, action]","[news_and_magazines, lifestyle, strategy, action]","[application, news_and_magazines]","[application, news_and_magazines]","[news_and_magazines, application]",0.20
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True,"[application, games, entertainment, strateg...",[application],"[game, strategy, action, adventure, casual, si...","[game, strategy, action, adventure, casual, si...","[games, adventure, casual]",0.00
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,"[games, game, entertainment, game puzzle, ...",[games],"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[games, puzzle, entertainment, simulation]",0.25
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[game, game_puzzle, action, adventure, casual]","[game, game_puzzle, action, adventure, casual]","[application, game, puzzle, adventure, casual,...","[application, game, puzzle, adventure, casual,...","[game_puzzle, game]",0.40
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, game]",0.40
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False,"[educational, , application, games, game, ente...","[application, games, game, entertainment]","[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_word, game]",0.20


In [47]:
df['jaccard_tool'] = df.apply(lambda r: jaccard(r.categories_normalized, r.openai_cats_tool_filtered), axis=1)
df

,bundle_id,title,description,store_url,category_names,ios,openai_cats_naive,openai_cats_naive_filtered,openai_cats_tool_filtered,openai_cats_tool,categories_normalized,jaccard_naive,jaccard_tool
32145,com.macys.android,Macy's,The latest version of the Macy’s app is better...,https://play.google.com/store/apps/details?id=...,"SHOPPING,APPLICATION",False,"[arcade, role_playing, education, casino, sports]",[],"[application, shopping, lifestyle]","[application, shopping, lifestyle]","[shopping, application]",0.00,0.666667
39423,com.pinterest,Pinterest,Pinterest is the place to explore inspiration....,https://play.google.com/store/apps/details?id=...,"LIFESTYLE,APPLICATION",False,"[lifestyle, shopping, casual, simulation, p]","[lifestyle, shopping, casual, simulation]","[application, lifestyle, shopping, adventure, ...","[application, lifestyle, shopping, adventure, ...","[lifestyle, application]",0.20,0.285714
508,1058528141,War Machines：Tanks Combat Game,"HELLO, COMMANDER ! \nWELCOME TO WAR MACHINES, ...",https://apps.apple.com/us/app/war-machines-tan...,"Games,Entertainment,Strategy,Action",True,[],[],"[application, games, game, strategy, action]","[application, games, game, strategy, action]","[games, entertainment, strategy, action]",0.00,0.500000
13106,com.breakingnewsbrief.app,US News - Breaking News Brief,Breaking News Brief delivers the world’s most ...,https://play.google.com/store/apps/details?id=...,"NEWS_AND_MAGAZINES,APPLICATION",False,"[news_and_magazines, lifestyle, strategy, action]","[news_and_magazines, lifestyle, strategy, action]","[application, news_and_magazines]","[application, news_and_magazines]","[news_and_magazines, application]",0.20,1.000000
4515,406889139,Coin Master,Join your Facebook friends and millions of pla...,https://apps.apple.com/us/app/coin-master/id40...,"Games,Adventure,Casual",True,"[application, games, entertainment, strateg...",[application],"[game, strategy, action, adventure, casual, si...","[game, strategy, action, adventure, casual, si...","[games, adventure, casual]",0.00,0.285714
5959,664575829,Fishdom,Never Fishdomed before? Then take a deep breat...,https://apps.apple.com/us/app/fishdom/id664575...,"Games,Puzzle,Entertainment,Simulation",True,"[games, game, entertainment, game puzzle, ...",[games],"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[games, puzzle, entertainment, simulation]",0.25,0.375000
56776,io.randomco.travel,Travel Town - Merge Adventure,"Explore Travel Town, where you can combine eve...",https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[game, game_puzzle, action, adventure, casual]","[game, game_puzzle, action, adventure, casual]","[application, game, puzzle, adventure, casual,...","[application, game, puzzle, adventure, casual,...","[game_puzzle, game]",0.40,0.125000
49136,com.tripledot.woodoku,Woodoku - Block Puzzle Games,Woodoku: a wood block puzzle game meets a sudo...,https://play.google.com/store/apps/details?id=...,"GAME_PUZZLE,GAME",False,"[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, puzzle, casual, strategy]","[game_puzzle, game]",0.40,0.200000
56467,in.playsimple.tripcross,Crossword Jam,Word Jam - A word search & word guess brain ga...,https://play.google.com/store/apps/details?id=...,"GAME_WORD,GAME",False,"[educational, , application, games, game, ente...","[application, games, game, entertainment]","[application, games, game, entertainment, game...","[application, games, game, entertainment, game...","[game_word, game]",0.20,0.222222
